# Welcome to the Prognostics Algorithms Package Tutorial

The goal of this notebook is to instruct the user on how to use and extend the NASA Python Prognostics Algorithms Package. 

First some background. The Prognostics Algorithms Package (`prog_algs`) contains tools for performing prognostics (event prediction) using the Prognostics Models Package. `prog_algs` also includes tools for analyzing the performance of prognostics algorithms. 

A few definitions:
* state estimation: The process of estimating the (possibly hidden) state of a system given sensor information on observable states
* prediction: The process of predicting the evolution of a system state with time and the occurance of events. 

The `prog_algs` package has the following structure
* `prog_algs/state_estimators/` - Tools for performing state estimation
* `prog_algs/predictors/` - Tools for performing prediction
* `prog_algs/metrics/` - Tools for analyzing the performance of prognostics algorithms
* `prog_algs/uncertain_data/` - Tools for representing data with uncertainty
* `prog_algs/visualize/` - Tools for visualizing results

In addition to the `prog_algs` package, this repo includes examples showing how to use the package (see `examples/`), a template for implementing a new state estimator (`state_estimator_template`), a template for implementing a new predictor (`predictor_template`), documentation (`docs/`), and this tutorial (`tutorial.ipynb`).

Before you start, install `prog_algs` using pip:
      `pip install prog_algs`
or, to use the pre-release, close from GitHub and checkout the dev branch. Then run the following command:
      `pip install -e .`

Now lets get started with some examples

## Prediction Example 
First thing to do is to import the prog_algs and the model you intend to use

In [ ]:
from prog_models.models.battery_circuit import BatteryCircuit
from prog_algs import *

Next, prepare the model like you did for simulation

In [ ]:
def future_loading(t, x={}):
    # Variable (piece-wise) future loading scheme 
    if (t < 600):
        i = 2
    elif (t < 900):
        i = 1
    elif (t < 1800):
        i = 4
    elif (t < 3000):
        i = 2
    else:
        i = 3
    return {'i': i}

batt = BatteryCircuit()

Now that we have our model ready, we can construct our state estimator:

In [ ]:
filt = state_estimators.UnscentedKalmanFilter(batt, batt.parameters['x0'])

The filter estimate function can then be run when there is updated data. Each iteration it will produce a new estimate of the system state (with uncertainty). For example:

In [ ]:
import matplotlib.pyplot as plt  # For plotting

print("Prior State:", filt.x.mean)
print('\tSOC: ', batt.event_state(filt.x.mean)['EOD'])
fig = filt.x.plot_scatter(label='prior')
example_measurements = {'t': 32.2, 'v': 3.915}
t = 0.1
filt.estimate(t, future_loading(t), example_measurements)
print("Posterior State:", filt.x.mean)
print('\tSOC: ', batt.event_state(filt.x.mean)['EOD'])
filt.x.plot_scatter(fig= fig, label='posterior')

fig.show()

That's the state estimation step- now lets prepare for prediction. 

In [ ]:
# Create predictor 
mc = predictors.MonteCarlo(batt)

# Generate Samples
samples = filt.x.sample(20)

Now lets use the constructed mc predictor to perform a single prediction. Here we're setting dt to 0.25. Note this may take up to a minute

In [ ]:
(times, inputs, states, outputs, event_states, toe) = mc.predict(samples, future_loading, dt=0.025)


The prediction returns the data corresponding to each sample and each save point. The format is `[sample #][time]` so that `states[m][n]` corresponds to the state for sample `m` at time `times[m][n]`

Next, let's use the metrics package to analyse the results

In [ ]:
print("\nEOD Predictions (s):")
from prog_algs.metrics import samples as metrics 
print('\tPercentage between 3005.2 and 3005.6: ', metrics.percentage_in_bounds(toe, [3005.2, 3005.6])*100.0, '%')
print('\tAssuming ground truth 3005.25: ', metrics.toe_metrics(toe, 3005.25))
print('\tP(Success) if mission ends at 3005.25: ', metrics.prob_success(toe, 3005.25))

You can also visualize the results in a variety of different ways. For example, state transition

In [ ]:
fig = states.snapshot(0).plot_scatter(label = "t={}".format(int(times[0][0])))
states.snapshot(10).plot_scatter(fig = fig, label = "t={}".format(int(times[0][10])))
states.snapshot(50).plot_scatter(fig = fig, label = "t={}".format(int(times[0][50])))

states.snapshot(-1).plot_scatter(fig = fig, label = "t={}".format(int(times[0][-1])))
fig.show()

## Extending - Adding a new state estimator
New state estimators can be created by extending the state_estimator interface. As an example lets use a really dumb state estimator that adds random noise each step - and accepts the state that is closest. 

First thing we need to do is import the StateEstimator parent class

In [ ]:
from prog_algs.state_estimators.state_estimator import StateEstimator

Next we select how state will be represented. In this case there's no uncertainty- it's just one state, so we represent it as a scaler. Import the appropriate class

In [ ]:
from prog_algs.uncertain_data import ScalarData

Now we construct the class, implementing the functions of the state estimator template (`state_estimator_template.py`)

In [ ]:
import random 

class BlindlyStumbleEstimator(StateEstimator):
    def __init__(self, model, x0):
        self.m = model
        self.state = x0

    def estimate(self, t, u, z):
        # Generate new candidate state
        x2 = {key : float(value) + 10*(random.random()-0.5) for (key,value) in self.state.items()}

        # Calculate outputs
        z_est = self.m.output(self.state)
        z_est2 = self.m.output(x2)

        # Now score them each by how close they are to the measured z
        z_est_score = sum([abs(z_est[key] - z[key]) for key in self.m.outputs])
        z_est2_score = sum([abs(z_est2[key] - z[key]) for key in self.m.outputs])

        # Now choose the closer one
        if z_est2_score < z_est_score: 
            self.state = x2

    @property
    def x(self):
        return ScalarData(self.state)


Great, now let's try it out using the model from earlier. with an initial state of all 0s. It should slowly converge towards the correct state

In [ ]:
x0 = {key: 0 for key in batt.states}
se = BlindlyStumbleEstimator(batt, x0)

for i in range(25):
    u = {'i': 0}
    z = {'t': 18.95, 'v': 4.183}
    se.estimate(i, u, z)
    print(se.x.mean)
    print("\tcorrect: {'tb': 18.95, 'qb': 7856.3254, 'qcp': 0, 'qcs': 0}")

## Conclusion
Thank you for trying out this tutorial. See the examples in the `examples/` folder for more details on how to use the package. Any questions can be directed to Chris Teubert (christopher.a.teubert@nasa.gov)